In [2]:
''' 
MULTI SMOTE LOG REG BINARY
'''
import warnings
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from sklearn.metrics import  accuracy_score, confusion_matrix,precision_score, recall_score, f1_score,roc_auc_score,roc_curve,precision_recall_curve, average_precision_score
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,RepeatedStratifiedKFold
from   Models import Multi_SMOTE_LogReg

'''
This return  the features found during the training
'''
import pandas as pd
def FoundFeatures(Path):
    # reading the found dataset for the training
    df = pd.read_excel(Path)
    Features = set(df.columns)
    return Features

# removing the id and the subtypes of a given dataset
def extract_PATIENT_INFO(db):
    ids =  db.pop("ID")
    morf_codificata = db.pop("morf_codificata")
    luogoTc = db.pop("luogoTC_codificato")
    return db, ids, morf_codificata, luogoTc

In [4]:
db = pd.read_excel("Dataset/Cleaned_dataset.xlsx")

 # extrating the label
y = db.pop("maligno")
X = db

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
best_params_per_fold = []
scores = pd.DataFrame(columns=['Outer Fold',"Grid-Search",'F1-score', 'Recall', 'Precision', 'Accuracy', 'Auc-Score', '#Features'])


train_folds = []
test_folds = []

for index, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]

    train_folds.append(X_train.copy())
    test_folds.append(X_test.copy())

    # configure the cross-validation procedure
    cv_inner = StratifiedKFold(n_splits=5,shuffle = True, random_state=42)
    pipeline = Multi_SMOTE_LogReg.Define_Pipeline()
    X_train, ids, morf_codificata, luogotcTRAIN = extract_PATIENT_INFO(X_train)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  
        Grid_searchOBJ = Multi_SMOTE_LogReg.Grid_Search(pipeline=pipeline,cv=cv_inner, X_train=X_train, y_train=y_train)

    best_params = Grid_searchOBJ.best_params_
    best_params_per_fold.append(Grid_searchOBJ.best_params_)
    print(f"Fold {len(best_params_per_fold)}: Best Params = {Grid_searchOBJ.best_params_}")

    ##### Elements/Multi_SMOTE_LogReg/AcrossSplits
    Chosen_pipeline = Multi_SMOTE_LogReg.Define_Pipeline()
    Chosen_pipeline.set_params(**Grid_searchOBJ.best_params_)
    fitted_pipeline = Multi_SMOTE_LogReg.fit(pipeline=Chosen_pipeline, X_train=X_train, y_train=y_train, Prefix="AcrossSplits/Split{}_".format(index))
    X_test, ids_test, morf_codificata_test, luogotcTEST = extract_PATIENT_INFO(X_test)
    
    predictions = fitted_pipeline.predict(X_test)
    probs = fitted_pipeline.predict_proba(X_test)
    
    ###Metrics
    Accuracy = round(accuracy_score(y_test, predictions),3)
    precision = round(precision_score(y_test, predictions),3)
    recall = round(recall_score(y_test, predictions),3)
    f1 = round(f1_score(y_test, predictions),3)
    probs = probs[:, 1]
    auc_roc = round(roc_auc_score(y_test,probs),3)
    print("TEST TIME F1 is ", f1)
	##Features
    AllFeaturesMULTI_SMOTE_LogReg = FoundFeatures("./Elements/Multi_SMOTE_LogReg/AcrossSplits/Split{}_Multi_SMOTE_LogReg_ReduceDataset.xlsx".format(index))
    print("it has found {} features".format(len(AllFeaturesMULTI_SMOTE_LogReg)))

    new_row = [index,Grid_searchOBJ.best_score_, f1, recall, precision, Accuracy, auc_roc, len(AllFeaturesMULTI_SMOTE_LogReg)]  
    scores.loc[index] = new_row  # Append the new row
    
scores["Outer Fold"] = scores["Outer Fold"].astype(int)
scores["#Features"] = scores["#Features"].astype(int)
scores.to_csv('Elements/Multi_SMOTE_LogReg/AcrossSplits/scores.csv', index=False)  # index=False to avoid saving the row numbers
scores



[2024-11-08 16:19:25,788: INFO: Multi_SMOTE_LogReg: Defining the Pipeline...]
Fitting 5 folds for each of 2700 candidates, totalling 13500 fits
Best: 0.790580 using {'LassoCoeffSelector__alpha': 0.004, 'logistic_regression__C': 1, 'logistic_regression__max_iter': 300, 'logistic_regression__penalty': 'l2', 'logistic_regression__solver': 'newton-cg', 'smote__k_neighbors': 2, 'smote__sampling_strategy': 'auto'}
Fold 1: Best Params = {'LassoCoeffSelector__alpha': 0.004, 'logistic_regression__C': 1, 'logistic_regression__max_iter': 300, 'logistic_regression__penalty': 'l2', 'logistic_regression__solver': 'newton-cg', 'smote__k_neighbors': 2, 'smote__sampling_strategy': 'auto'}
[2024-11-08 16:34:26,270: INFO: Multi_SMOTE_LogReg: Defining the Pipeline...]
Time taken to fit the model: 0.83262 seconds
[2024-11-08 16:34:27,250: INFO: Multi_SMOTE_LogReg: saving the resulting dataframe into: Multi_SMOTE_LogReg_ReduceDataset.xlsx]
TEST TIME F1 is  0.735
it has found 14 features
[2024-11-08 16:34:27

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.790580,0.735,0.857,0.643,0.876,0.954,14
1,1,0.785989,0.745,0.927,0.623,0.876,0.965,4
2,2,0.783237,0.778,0.833,0.729,0.904,0.964,24
3,3,0.804804,0.740,0.881,0.638,0.876,0.933,11
4,4,0.785042,0.800,0.905,0.717,0.909,0.959,4


In [5]:
best_params_per_fold

[{'LassoCoeffSelector__alpha': 0.004,
  'logistic_regression__C': 1,
  'logistic_regression__max_iter': 300,
  'logistic_regression__penalty': 'l2',
  'logistic_regression__solver': 'newton-cg',
  'smote__k_neighbors': 2,
  'smote__sampling_strategy': 'auto'},
 {'LassoCoeffSelector__alpha': 0.106,
  'logistic_regression__C': 1,
  'logistic_regression__max_iter': 300,
  'logistic_regression__penalty': 'l2',
  'logistic_regression__solver': 'liblinear',
  'smote__k_neighbors': 2,
  'smote__sampling_strategy': 'auto'},
 {'LassoCoeffSelector__alpha': 0.004,
  'logistic_regression__C': 10,
  'logistic_regression__max_iter': 300,
  'logistic_regression__penalty': 'l2',
  'logistic_regression__solver': 'newton-cg',
  'smote__k_neighbors': 5,
  'smote__sampling_strategy': 'auto'},
 {'LassoCoeffSelector__alpha': 0.01,
  'logistic_regression__C': 10,
  'logistic_regression__max_iter': 300,
  'logistic_regression__penalty': 'l2',
  'logistic_regression__solver': 'newton-cg',
  'smote__k_neighbors

In [6]:
import numpy as np
def compute_stats(scores):
    cols = scores.columns[1:-1]
    mean_row_values = scores[cols].mean()
    std_row_values = scores[cols].std()
    
    # Create new rows with NaNs for the columns that will be replaced
    mean_row = pd.Series("", index=scores.columns, name='mean')
    std_row = pd.Series("", index=scores.columns, name='std')
    mean_row[cols] = mean_row_values.round(3)
    std_row[cols] = std_row_values.round(3)
    
    # Convert Series to DataFrame
    mean_df = mean_row.to_frame().T
    std_df = std_row.to_frame().T
    
    scores_new = pd.concat([scores, mean_df, std_df], axis = 0)
    
    scores_new.loc["mean","#Features"] = str(int(round(np.mean(scores_new["#Features"][:-2]),0)))
    scores_new.loc["std","#Features"] = str(int(round(np.std( scores_new["#Features"][:-2]),0)))
    
    scores_new.to_csv('Elements/Multi_SMOTE_LogReg/AcrossSplits/scores.csv', index=False)  # index=False to avoid saving the row numbers
    
    return scores_new

scores_agg = compute_stats(scores)
scores_agg

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.79058,0.735,0.857,0.643,0.876,0.954,14
1,1,0.785989,0.745,0.927,0.623,0.876,0.965,4
2,2,0.783237,0.778,0.833,0.729,0.904,0.964,24
3,3,0.804804,0.74,0.881,0.638,0.876,0.933,11
4,4,0.785042,0.8,0.905,0.717,0.909,0.959,4
mean,,0.79,0.76,0.881,0.67,0.888,0.955,11
std,,0.009,0.028,0.037,0.049,0.017,0.013,7


In [7]:
import matplotlib.pyplot as plt
def StackedHist(df):
    # Aggregate counts by Center and Morf_Codificata
    counts = df.groupby(['luogoTC_codificato', 'morf_codificata']).size().unstack(fill_value=0)

    # Define all possible center labels (1 to 14)
    all_centers = list(range(1, 15))
    all_labels = sorted(set(map(int,df['morf_codificata'])))
    
    # Reindex the DataFrame to include all centers, filling missing values with 0
    counts = counts.reindex(index=all_centers, columns=all_labels, fill_value=0)

    # Create the stacked histogram
    fig, ax = plt.subplots(figsize=(12, 10))

    # Initialize bottom positions for stacking
    bottom = np.zeros(len(all_centers))
    
    
    colors = plt.cm.tab20.colors



    # Plot each Morf_Codificata category as a stacked bar segment
    for i, label in enumerate(all_labels):
        ax.bar(all_centers, counts[label], bottom=bottom, label=f'{label}', color=colors[i])
        bottom += counts[label].values  # Update bottom for next stack

    # Set x-ticks from 1 to 14
    ax.set_xticks(all_centers)
    ax.set_xticklabels(all_centers, rotation=45, ha='right')

    # Adding labels and title
    plt.xlabel('Center')
    plt.ylabel('Count')
    plt.title('Stacked Histogram by Center and Morf_Codificata')
    plt.legend(title='Morf_Codificata', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()